# Energy Detection

## Energy Detection Function

In [54]:
import numpy as np

def energy_detection_array(sig: np.ndarray, threshold: float):
    """
    Energy detection from a float16 array where the last axis holds [real, imag].

    Parameters
    ----------
    sig : np.ndarray
        A NumPy array of dtype=float16 whose last axis has size 2.
        In other words, each “complex sample” is stored as:
            sig[..., 0] = real part (float16)
            sig[..., 1] = imag part (float16)
        You can have any number of leading dimensions (e.g. antennas × samples),
        but the final dimension must be exactly 2.

    threshold : float
        The energy threshold. If the computed average energy exceeds this value,
        the function returns True (signal detected); otherwise False.

    Returns
    -------
    decision : bool
        True if (average energy > threshold), otherwise False.

    energy : float
        The computed average energy (as a Python float).

    Raises
    ------
    TypeError
        - If `sig` is not a NumPy array.
        - If `sig.dtype` is not `np.float16`.
        - If the last axis of `sig` is not exactly length 2.
    """

    # 1) Check that input is a NumPy array
    if not isinstance(sig, np.ndarray):
        raise TypeError("`sig` must be a numpy.ndarray.")

    # 2) Check dtype is float16
    if sig.dtype != np.float16:
        raise TypeError(
            f"`sig.dtype` must be float16, but got {sig.dtype}."
        )

    # 3) Check that the last axis has length 2
    if sig.shape[1] != 2:
        raise TypeError(
            "Axis=1 of `sig` must have length 2 (i.e. [real, imag]), "
            f"but got sig.shape[1] = {sig.shape[1]}."
        )


    # 4) Separate out real and imag parts (both still float16)
    #    Cast each to float32 before squaring to reduce overflow/underflow risk.
    real_part = sig.take(indices=0, axis=1)  # dtype=float16
    imag_part = sig.take(indices=1, axis=1)  # dtype=float16

    # 5) Compute sum of squared magnitudes: sum(real^2 + imag^2)
    energy_sum = np.sum(real_part**2 + imag_part**2, dtype=np.float32)

    # 6) Number of complex samples = total_size_of_array / 2
    #    (since each pair [real, imag] occupies 2 float16 slots)
    num_complex_samples = sig.size // 2

    # 7) Average energy
    energy = energy_sum / num_complex_samples

    # 8) Compare with threshold
    decision = bool(energy > threshold)
    return decision, float(energy)


## Energy Detection Example

In [55]:
np.random.seed(0)
real_part = (np.random.randn(256) * 0.1).astype(np.float16)
imag_part = (np.random.randn(256) * 0.1).astype(np.float16)
sig = np.stack([real_part, imag_part], axis=1)

threshold = 0.005

decision, energy = energy_detection_array(sig, threshold)
print(f"Detection result: {decision}")
print(f"Computed average energy: {energy:.6f}")

print(sig.shape)

Detection result: True
Computed average energy: 0.019970
(256, 2)


# Kurtosis Detection

## Kurtosis Detection Function

In [62]:
import numpy as np

def kurtosis_detection_array_float16(sig: np.ndarray, threshold: float):
    """
    Kurtosis-based detection using only float16 arithmetic,
    from an array where the last axis holds [real, imag].

    Parameters
    ----------
    sig : np.ndarray
        A NumPy array of dtype=float16 whose last axis has size 2.
        Each “complex sample” is stored as:
            sig[..., 0] = real part (float16)
            sig[..., 1] = imag part (float16)
        Any number of leading dimensions is allowed (e.g., antennas × samples),
        but the final dimension must be exactly 2.

    threshold : float
        Kurtosis threshold. If the computed kurtosis (in float16) exceeds this value,
        the function returns True (signal detected); otherwise False.

    Returns
    -------
    decision : bool
        True if computed kurtosis > threshold, otherwise False.

    kurtosis : np.float16
        The computed kurtosis metric in float16.

    Raises
    ------
    TypeError
        - If `sig` is not a NumPy array.
        - If `sig.dtype` is not `np.float16`.
        - If the last axis of `sig` is not exactly length 2.
    """

    # 1) Check that input is a NumPy array
    if not isinstance(sig, np.ndarray):
        raise TypeError("`sig` must be a numpy.ndarray.")

    # 2) Check dtype is float16
    if sig.dtype != np.float16:
        raise TypeError(
            f"`sig.dtype` must be float16, but got {sig.dtype}."
        )

    # 3) Check that the last axis has length 2
    if sig.shape[-1] != 2:
        raise TypeError(
            "Last axis of `sig` must have length 2 (i.e. [real, imag]), "
            f"but got last axis = {sig.shape[-1]}."
        )

    # 4) Separate out real and imag parts (both float16)
    # real_part = sig[..., 0]    # dtype=float16
    # imag_part = sig[..., 1]    # dtype=float16
    real_part = sig.take(indices=0, axis=1)  # dtype=float16
    imag_part = sig.take(indices=1, axis=1)  # dtype=float16

    # 5) Compute instantaneous power p = real^2 + imag^2, all in float16
    power = real_part * real_part + imag_part * imag_part  # dtype=float16

    # 6) Number of complex samples = total_size_of_array // 2
    num_complex_samples = sig.size // 2  # integer

    # 7) Compute second moment E2 = mean(power) in float16
    energy_sum = np.sum(power, dtype=np.float16)           # float16 sum
    E2 = energy_sum / np.float16(num_complex_samples)      # float16

    # 8) Compute fourth moment E4 = mean(power^2) in float16
    power_sq = power * power                                # float16
    fourth_sum = np.sum(power_sq, dtype=np.float16)         # float16 sum
    E4 = fourth_sum / np.float16(num_complex_samples)       # float16

    # 9) Compute kurtosis = E4 / (E2^2) in float16
    if E2 == np.float16(0.0):
        kurtosis = np.float16(0.0)
    else:
        kurtosis = E4 / (E2 * E2)  # float16 division

    # 10) Compare with threshold (threshold is cast to float16)
    decision = bool(kurtosis > np.float16(threshold))
    return decision, kurtosis


## Kurtosis Example

In [64]:
# 1) Create separate float16 arrays for real & imag (1D signal of length 256)
np.random.seed(0)
real_part = (np.random.randn(256) * 0.1).astype(np.float16)  # shape = (256,)
imag_part = (np.random.randn(256) * 0.1).astype(np.float16)  # shape = (256,)

# 2) Stack real & imag into a single array with last axis=2
sig = np.stack([real_part, imag_part], axis=1)  # shape = (256, 2), dtype=float16

# 3) Define a kurtosis threshold (as float; will be cast to float16 inside)
threshold = 3.0

# 4) Run kurtosis detection entirely in float16
decision, kurtosis_value = kurtosis_detection_array_float16(sig, threshold)

print(f"Detection result: {decision}, threshold {np.float16(threshold)}")
print(f"Computed kurtosis (float16): {kurtosis_value}")

Detection result: False, threshold 3.0
Computed kurtosis (float16): 1.82421875


# Chirp Detection

## Chirp Detection Function

In [8]:
import numpy as np

def chirp_detection_axis1_float16(sig: np.ndarray, fs: float, ndelay: int):
    """
    Chirp detection using delayed‐self multiplication and a manual DFT,
    with every arithmetic step performed in float16.

    Input format: a float16 NumPy array of shape (n_samples, 2),
    where axis=1 holds [real, imag] for each complex sample.

    Parameters
    ----------
    sig : np.ndarray
        A NumPy array of dtype=float16 and shape (n_samples, 2).
        Each row represents one complex sample:
            sig[t, 0] = real part (float16)
            sig[t, 1] = imag part (float16)
        There is exactly one “antenna,” so the first dimension is n_samples.
    fs : float
        Sampling frequency in Hz. This will be cast to float16 internally.
    ndelay : int
        Delay in samples. Must satisfy 1 ≤ ndelay < n_samples.

    Returns
    -------
    chirp_rate : np.float16
        Estimated chirp rate in Hz/s (frequency shift per second), in float16.
    freq_at_peak : np.float16
        Frequency (in Hz) at which the FFT‐spectrum of (z·conj(z_delayed)) peaks, in float16.
    peak_value : np.float16
        Magnitude of that FFT bin at the peak, in float16.
    peak_index : int
        Index (0..n_samples−1) of the peak bin in the FFTshifted spectrum.

    Raises
    ------
    TypeError
        - If `sig` is not a NumPy array.
        - If `sig.dtype` is not np.float16.
        - If `sig` does not have shape (n_samples, 2).
    ValueError
        - If `ndelay` is not in the range 1 ≤ ndelay < n_samples.
    """

    # 1) Type checks
    if not isinstance(sig, np.ndarray):
        raise TypeError("`sig` must be a numpy.ndarray.")
    if sig.dtype != np.float16:
        raise TypeError(f"`sig.dtype` must be float16, but got {sig.dtype}.")
    if sig.ndim != 2 or sig.shape[1] != 2:
        raise TypeError("`sig` must have shape (n_samples, 2), with axis=1 = [real, imag].")

    n_samples = sig.shape[0]
    if not (1 <= ndelay < n_samples):
        raise ValueError("`ndelay` must satisfy 1 ≤ ndelay < n_samples.")

    # 2) Cast fs and n_samples to float16 for all downstream arithmetic
    fs16 = np.float16(fs)
    N16 = np.float16(n_samples)

    # 3) Split real/imag (both float16)
    real16 = sig[:, 0]  # shape = (n_samples,), dtype=float16
    imag16 = sig[:, 1]  # shape = (n_samples,), dtype=float16

    # 4) Build delayed signals (zero‐pad first ndelay samples)
    #    z_delay[t] = z[t - ndelay] for t ≥ ndelay, else 0
    real_delay = np.zeros(n_samples, dtype=np.float16)
    imag_delay = np.zeros(n_samples, dtype=np.float16)
    real_delay[ndelay:] = real16[: n_samples - ndelay]
    imag_delay[ndelay:] = imag16[: n_samples - ndelay]

    # 5) Compute product: prod = z * conj(z_delay), in pure float16
    #    If z = a + j b, z_delay = c + j d, conj(z_delay) = c - j d, then
    #      prod_real[n] = a·c + b·d
    #      prod_imag[n] = b·c - a·d
    prod_real = np.zeros(n_samples, dtype=np.float16)
    prod_imag = np.zeros(n_samples, dtype=np.float16)
    for n in range(n_samples):
        a = real16[n]             # float16
        b = imag16[n]             # float16
        c = real_delay[n]         # float16
        d = imag_delay[n]         # float16

        # a·c, b·d, b·c, a·d each computed in float16
        ac = np.float16(a * c)
        bd = np.float16(b * d)
        bc = np.float16(b * c)
        ad = np.float16(a * d)

        prod_real[n] = np.float16(ac + bd)
        prod_imag[n] = np.float16(bc - ad)

    # 6) Manual DFT (no external FFT) fully in float16:
    #    X[k] = Σ_{n=0..N−1} [ prod[n] · exp(−j·2π·n·k/N ) ]
    #    Let θ = −2π·n·k / N.  Then exp(−jθ) = cos(θ) − j sin(θ).
    #    If prod[n] = a + j b, then:
    #      term_real = a·cos(θ) + b·sin(θ)
    #      term_imag = b·cos(θ) − a·sin(θ)
    X_real = np.zeros(n_samples, dtype=np.float16)
    X_imag = np.zeros(n_samples, dtype=np.float16)
    two_pi = np.float16(2.0 * np.pi)

    for k in range(n_samples):
        sum_r = np.float16(0.0)
        sum_i = np.float16(0.0)
        # pre‐compute k/N in float16
        k_over_N = np.float16(k) / N16
        for n in range(n_samples):
            # θ = −2π · (n·k/N), all in float16
            angle = np.float16(-1.0) * two_pi * (np.float16(n) * k_over_N)
            cos_a = np.cos(angle)  # float16
            sin_a = np.sin(angle)  # float16

            a = prod_real[n]       # float16
            b = prod_imag[n]       # float16

            # term_real = a·cos(θ) + b·sin(θ)
            t_r = np.float16(a * cos_a) + np.float16(b * sin_a)
            # term_imag = b·cos(θ) − a·sin(θ)
            t_i = np.float16(b * cos_a) - np.float16(a * sin_a)

            sum_r = np.float16(sum_r + t_r)
            sum_i = np.float16(sum_i + t_i)

        X_real[k] = sum_r
        X_imag[k] = sum_i

    # 7) FFT shift: rotate by n_samples/2 so negative freqs are up front
    half = n_samples // 2
    Xr_shift = np.concatenate((X_real[half:], X_real[:half]))
    Xi_shift = np.concatenate((X_imag[half:], X_imag[:half]))

    # 8) Magnitude spectrum |X| = sqrt(Real^2 + Imag^2), all in float16
    mag = np.sqrt(np.float16(Xr_shift * Xr_shift + Xi_shift * Xi_shift))

    # 9) Find peak index (Python int) and value (float16)
    peak_index = int(np.argmax(mag))
    peak_value = mag[peak_index]  # float16

    # 10) Build frequency axis (float16):
    #     freqs[k] = (k - N/2)/N * fs
    kvec = np.arange(n_samples, dtype=np.float16)
    shift = kvec - np.float16(half)        # dtype=float16
    freqs = shift * (fs16 / N16)           # dtype=float16
    freq_at_peak = freqs[peak_index]       # float16

    # 11) Time corresponding to the chosen delay (float16)
    t_elapsed = np.float16(ndelay) / fs16  # float16

    # # 12) Chirp rate = freq_at_peak / t_elapsed (all float16)
    # if t_elapsed == np.float16(0.0):
    #     chirp_rate = np.float16(0.0)
    # else:
    #     chirp_rate = np.float16(freq_at_peak / t_elapsed)

    max_f16 = np.finfo(np.float16).max

    with np.errstate(divide='ignore', over='ignore'):
        raw_rate = np.float16(freq_at_peak / t_elapsed)

    # If division overflowed to ±inf or produced NaN, saturate:
    if np.isinf(raw_rate) or np.isnan(raw_rate):
        # Keep the sign, and clamp to ±max_f16
        if raw_rate > np.float16(0.0):
            chirp_rate = np.float16(max_f16)
        else:
            chirp_rate = np.float16(-max_f16)
    else:
        chirp_rate = raw_rate

    return chirp_rate, freq_at_peak, peak_value, peak_index


## Chirp Detection Example 1

In [9]:
import numpy as np

# 1) Parameters
fs = 20_000.0    # 20 kHz sampling rate
n = 256
t = np.arange(n) / fs

# Generate a test linear chirp from 1 kHz → 3 kHz over n=256 samples
f0 = 1_000.0
f1 = 3_000.0
k_rate = (f1 - f0) / (n / fs)          # true chirp rate in Hz/s
phase = 2 * np.pi * (f0*t + 0.5*k_rate*t**2)
z = np.cos(phase) + 1j * np.sin(phase)  # complex64 by default

# 2) Convert to float16 and stack into shape (n, 2)
real_part = z.real.astype(np.float16)   # (256,)
imag_part = z.imag.astype(np.float16)   # (256,)
sig = np.stack([real_part, imag_part], axis=1)  # (256, 2), dtype=float16

# 3) Choose a delay, e.g. ndelay = 50 samples
ndelay = 50

# 4) Run chirp detection (all arithmetic in float16)
chirp_rate_est, freq_peak, mag_peak, idx_peak = (
    chirp_detection_axis1_float16(sig, fs, ndelay)
)

print(f"Estimated chirp rate: {chirp_rate_est} Hz/s (float16)")
print(f"Peak frequency: {freq_peak} Hz (float16)")
print(f"Peak magnitude: {mag_peak} (float16)")
print(f"Peak index: {idx_peak}")


Estimated chirp rate: -65504.0 Hz/s (float16)
Peak frequency: -390.5 Hz (float16)
Peak magnitude: 199.5 (float16)
Peak index: 123


## Chirp Detection Example 2

In [10]:
import numpy as np

# ------------------------------------------------------------
# Example: Slow Linear Chirp (100 Hz → 200 Hz over 256 samples)
# ------------------------------------------------------------

# 1) Parameters
fs = 1000.0         # 1 kHz sampling rate
n = 256             # number of samples
t = np.arange(n) / fs  # time vector from 0 to (n−1)/fs

# 2) Design a linear chirp: f(t) = f0 + k·t,  where k = (f1−f0)/(T)
f0 = 100.0          # start frequency 100 Hz
f1 = 200.0          # end frequency 200 Hz
T  = n / fs         # total chirp duration = 256/1000 = 0.256 s
k  = (f1 - f0) / T  # slope in Hz/s

# instantaneous phase: φ(t) = 2π·(f0·t + ½·k·t²)
phase = 2.0 * np.pi * ( f0 * t + 0.5 * k * t**2 )

# 3) Build the complex‐valued chirp and convert to float16 [shape = (256,)]
z = np.cos(phase) + 1j * np.sin(phase)   # by default complex64

real_part = z.real.astype(np.float16)     # shape = (256,), dtype=float16
imag_part = z.imag.astype(np.float16)     # shape = (256,), dtype=float16

# 4) Pack into a float16 array of shape (256, 2), axis=1 = [real, imag]
sig = np.stack([real_part, imag_part], axis=1)  # shape = (256, 2), dtype=float16

# 5) Choose a small delay so freq‐shift remains small
ndelay = 10   # e.g. 10 samples → t_elapsed = 10/1000 = 0.01 s

# 6) Call the chirp‐detection routine (all float16 internally)
chirp_rate_est, freq_peak, mag_peak, idx_peak = \
    chirp_detection_axis1_float16(sig, fs, ndelay)

print("Estimated chirp rate: ", chirp_rate_est, "Hz/s (float16)")
print("Frequency at peak:     ", freq_peak,    "Hz (float16)")
print("Peak magnitude:        ", mag_peak,      "(float16)")
print("Peak index:            ", idx_peak,      "(int)")


Estimated chirp rate:  -390.5 Hz/s (float16)
Frequency at peak:      -3.906 Hz (float16)
Peak magnitude:         235.4 (float16)
Peak index:             127 (int)
